# BLOCKCHAIN CLASS MODULE
The heart of it all.

In [1]:
#IMPORTS
import numpy as np

## BLOCK CLASS

A block has four attributes:
* Its height in the blockchain.
* The height of its parent.
* The player who mined it. This is equivalent to the value of the assignment map $\iota$ for this block.

Additionally, we have a method to print the attributes in a convenient form.

In [2]:
class Block:
    def __init__(self, _h, _p, _i):
        self.height = _h #equivalent to the period where the block was mined
        self.parent = _p #height of parent block
        self.iota = _i #index of the miner who mined the block

    def printBlock(self):
        print("block", self.height, "mined by miner", f"{self.iota},", "parent is block", self.parent)

## MINER CLASS

A miner has two attributes:
* The index of the miner. In our analysis this is always either $0$ or $1$.
* An array that keeps track of all the stages this miner won.

The only method prints the attributes in an easily readable form.

In [3]:
class Miner:
    def __init__(self, _T, _i):
        #
        #'_T' is the number of stages in the finite blockchain game
        #'_i' is the index of the miner
        #
        self.index = _i
        self.winner = np.full(_T, False) #the value of 'self.winner[t]' is 'True' if and only if the miner won that stage 't'

    def printMiner(self):
        stages_won = np.arange(1, self.winner.shape[0] + 1)[self.winner]
        print("miner", self.index, "wins stages", stages_won)

## BLOCKCHAIN CLASS
This class builds on the above classes. A blockchain has the following attributes:
* The number of players. In our case this is always equal to $2$.
* An array that keeps track of the parent-child relation $\Lleftarrow$ on the sequence of blocks.
* An array that keeps track of the winners of each block. This is equivalent to the assignment map $\iota$.
* The number of stages $T$ the game has.
* An array that contains $n$ objects of class 'Miner'. The method $\_\_assignMiners()$ is responsible for the creation of this array.
* An array that contains $T$ objects of class 'Block'. The method $\_\_generateChain()$ is responsible for the creation of this array.
* Lastly, we have a list of the longest chains. This is done because it is more efficient. We construct the list when the Blockchain object is created. Later we can simply look up what the longest chains are, instead of computing them each time we need that information. The method $\_\_getLongestChains()$ is responsible for the creation of this list.

NOTE: Some methods' start with double underscores '$\_\_$ , this is supposed to signify that these methods should only ever be called by the class itself. Python lacks the private / public functionality we know from other languages.

### REMAINING METHODS

The remaining methods not yet mentioned are straight forward. The method $getPayoff()$ is used to calculate the number of blocks a miner $i$ mined in a chain ending in block $b_t$. The second method $chainLength()$ is used to calculate the length of the chain ending in the block $b_t$.

### CONVENIENT PROPERTY OF BLOCKCHAINS

We make use of the fact that a block can never have more than one parent. This allows us to iterate backwards all the way from a given block $b_t$ to the genesis block $b_0$. This is a nice and convenient property of blockchains. The path from block $b_t$ to $b_0$ were not necessarily unique if blocks were allowed multiple parents.

In [3]:
class Blockchain:
    def __init__(self, _n, _parents, _winners):
        #
        #'_parents' is an np.array where '_structure[t]' is the parent of block 't+1', e.g. the fifth element is the index of the parent of the block in t=5
        #'_winners' is an np.array where '_winners[t]' is the index of the winner of stage 't+1'
        #
        #'self.sequence' is an array with objects of class 'Block'
        #'self.horizon' is the time horizon of the game and is implicitly given.
        #'self.miners' is an array filled with objects of class 'Miner'
        #'self.winners' is an array with the index of the winning miner of every stage.
        #

        self.n = _n
        self.parents = _parents
        self.winners = _winners
        self.horizon = self.parents.shape[0]

        self.miners = self.__assignMiners()
        self.sequence = self.__generateChain()

        #pre-calculate a list of the longest chains for more speed
        self.longestchains = self.__getLongestChains() #a list of lists



    def __assignMiners(self):
        #
        #construct an array filled with objects of class 'Miner'
        #assign to each miner the blocks they won
        #

        #CREATE ARRAY
        miners = np.array([])
        for i in range(self.n):
            miners = np.append(miners, Miner(self.horizon, i))

        #ASSIGN BLOCKS
        for miner in miners:
            for t in range(self.winners.shape[0]):
                if miner.index == self.winners[t]:
                    miner.winner[t] = True

        return miners



    def __generateChain(self):
        #
        #generates the chain according to the miners strategies and the rounds they win
        #

        #CREATE ARRAY
        sequence = [] #use a list because appending to lists is approximately 5x faster than appending to numpy arrays
        for t in range(self.horizon + 1): #fill a chain with default blocks, we have 'T+1' blocks: that is 'T' blocks mined plus the genesis block
            if t == 0:
                sequence.append(Block(0, np.nan, np.nan)) #genesis block
            else:
                sequence.append(Block(t, self.parents[t-1], self.winners[t-1])) 

        sequence = np.array(sequence) #convert to array

        return sequence



    def __getLongestChains(self): 
        #
        #returns a list of lists filled with the intermediate stages 't' where the blocks in the longest chains were mined
        #example: if the longest chain is '{b0, b1, b2, b4, b6, b9}' it returns a list '[[0, 1, 2, 4, 6, 9]]'
        #we only ever need to know the longest chain at the end of the game, not in intermediate stages. we take advantage of this fact.
        #

        T = self.horizon
        longest_chains = []
        longest_chain_length = 0

        for t in range(T+1):
            #test if the block 't' has a child, if not, then it is the last block in a chain
            #we take advantage of the fact that chains that end at blocks with a child can never be the longest

            childless = True

            for s in range(t+1, T+1): #for each block that could be child of block 't'
                if self.sequence[s].parent == t: #if this is true, then block 't' has a child
                    childless = False
                    break

            if childless == True: #if block 't' is childless
                candidate_chain = [0] #the longest chain always starts with the genesis block at height 0
                u = t #'u' is the index of the block we are currently looking at, we start with the last block in the chain: block 't'

                #ITERATE BACKWARDS
                #keep appending blocks to the candidate chain as long as we have not reached the genesis block with height 0
                while self.sequence[u].height != 0: 
                    candidate_chain.append(u)
                    u = self.sequence[u].parent

                length_candidate_chain = len(candidate_chain)

                #TEST LENGTH
                if length_candidate_chain >= longest_chain_length: #if the chain is one of the longest we've seen so far
                    candidate_chain.sort()

                    #ADD
                    #add the candidate chain to the list of longest chains if it is not strictly the longest
                    if length_candidate_chain == longest_chain_length:
                        longest_chains.append(candidate_chain)

                    #OR REPLACE
                    #replace the list of longest chains with the candidate chain if it is strictly longer than all chains we saw before
                    if length_candidate_chain > longest_chain_length: 
                        longest_chain_length = length_candidate_chain
                        longest_chains = [candidate_chain]

        return longest_chains



    def getPayoff(self, _i, _t):
        #
        #sums up all the blocks miner '_i' mined in the chain going from block 0 to block '_t'
        #returns an integer corresponding to the payoff if the game were to end in '_t'
        #

        payoff = 0
        t = _t

        #ITERATE BACKWARDS
        while self.sequence[t].height != 0:
            if self.winners[t-1] == _i:
                payoff += 1
            t = self.sequence[t].parent

        return payoff


    def chainLength(self, _t):
        #
        #returns the length of the chain ending in block mined in stage '_t'
        #

        length = 1 #we must start at 1 because the genesis block is not counted in the while loop below
        t = _t

        #ITERATE BACKWARDS
        while self.sequence[t].height != 0:
            length += 1
            t = self.sequence[t].parent

        return length

## BLOCKCHAIN EXTENSION CLASS

The blockchain is extended often. The simplest solution we found is to create a new class for this purpose.

In [5]:
class ExtendedBlockchain(Blockchain):
    def __init__(self, _B, _i, _t):
        #
        #an extended copy of the base blockchain '_B', extended by one block, which is appended at block at height '_t'
        #miner '_i' wins the appended block
        #

        #EXTEND
        parents = np.append(_B.parents, _t)
        winners = np.append(_B.winners, _i)

        #BUILD
        Blockchain.__init__(self, _B.n, parents, winners)